<a href="https://colab.research.google.com/github/afairley/ColaboratoryNotebooks/blob/main/JaxBuildingMatrixAndTensorOperationsWithVMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
jax.print_environment_info(return_string=False)

jax:    0.4.23
jaxlib: 0.4.23
numpy:  1.25.2
python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
jax.devices (1 total, 1 local): [CpuDevice(id=0)]
process_count: 1


In [2]:
import jax.numpy as jnp
from jax import vmap

vv = lambda x, y: jnp.vdot(x, y)  #  ([a], [a]) -> []
mv = vmap(vv, (0, None), 0)      #  ([b,a], [a]) -> [b]      (b is the mapped axis)
mm = vmap(mv, (None, 1), 1)      #  ([b,a], [a,c]) -> [b,c]  (c is the mapped axis)
mv1 = vmap(vv, (0, 0), 0)   #  ([b,a], [b,a]) -> [b]        (b is the mapped axis)
mv2 = vmap(vv, (0, 1), 0)   #  ([b,a], [a,b]) -> [b]        (b is the mapped axis)
mm2 = vmap(mv2, (1, 1), 0)  #  ([b,c,a], [a,c,b]) -> [c,b]  (c is the mapped axis)

In [6]:
identityMatrix = ((1,0),
                  (0,1))
#expansionMatrix = map( lambda x: map( lambda y: y *2, x) *2 , identityMatrix)
#You can no longer do the above.

TypeError: 'map' object is not callable

In [ ]:
#Doing this isn't actually needed as colab instances
#come with jax installed
projects = [
    ("jax","https://github.com/google/jax")]
JAX_DIR = f"/content/{projects[0][0]}"
for project, repo in projects:
  !rm -rf ./{project}
  !git clone {repo}

In [ ]:
#Doing this isn't actually needed as colab instances
#come with jax installed
!pip install jaxlib
!cd {JAX_DIR} && pip install -e .
